In [13]:
framecount =0

# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2
# Import Pandas for save depth 
import pandas

# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)


# Streaming loop
try:
    while True:
        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()
        # frames.get_depth_frame() is a 640x360 depth image

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        framecount = framecount+1
        print("Frame : ",framecount)
        print("Size : ",len(depth_image[0])," X ",len(depth_image))
        print(depth_image)
        print()

        # Remove background - Set pixels further than clipping_distance to grey
        grey_color = 153
        depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
        bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)

        # Render images
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((bg_removed, depth_colormap))
        cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Example', images)
        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()


Depth Scale is:  0.0010000000474974513
Frame :  1
Size :  640  X  480
[[   0    0    0 ... 1895 1895 1895]
 [   0    0    0 ... 1895 1895 1895]
 [   0    0    0 ... 1930 1930 1924]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  2
Size :  640  X  480
[[   0    0    0 ...    0    0 1712]
 [   0    0    0 ...    0 1671 1671]
 [   0    0    0 ... 1640 1640 1649]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  3
Size :  640  X  480
[[   0    0    0 ... 1756 1756 1761]
 [   0    0    0 ... 1756 1756 1761]
 [   0    0    0 ... 1776 1776 1781]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  4
Size :  640  X  480
[[   0    0    0 ... 1817 1812 1812]
 [   0    0    0 ... 1817 1812 1812]
 [   0    0    0 ...    0 1817 1817]
 ...
 [   0    0    0 ...    0    0    

Frame :  37
Size :  640  X  480
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Frame :  38
Size :  640  X  480
[[   0    0    0 ... 1817 1817 1823]
 [   0    0    0 ... 1817 1817 1823]
 [   0    0    0 ... 1828 1828 1828]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  39
Size :  640  X  480
[[   0    0    0 ...    0    0    0]
 [   0    0    0 ... 1966    0    0]
 [   0    0    0 ... 1930    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  40
Size :  640  X  480
[[   0    0    0 ... 1855 1855 1855]
 [   0    0    0 ... 1844 1850 1850]
 [   0    0    0 ... 1844 1844 1844]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  41
Size :  640  X  480
[[   0    0    0 ... 2030 2030 2

Frame :  74
Size :  640  X  480
[[   0    0    0 ... 1883 1883 1883]
 [   0    0    0 ... 1883 1883 1883]
 [   0    0    0 ... 1895 1895 1895]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  75
Size :  640  X  480
[[   0    0    0 ... 1796 1796 1796]
 [   0    0    0 ... 1796 1796 1796]
 [   0    0    0 ... 1796 1796 1801]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  76
Size :  640  X  480
[[   0    0    0 ... 1948 1948 1948]
 [   0    0    0 ... 1936 1942 1942]
 [   0    0    0 ... 1912 1912 1912]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  77
Size :  640  X  480
[[   0    0    0 ... 1791 1791 1791]
 [   0    0    0 ... 1791 1791 1791]
 [   0    0    0 ... 1791 1791 1791]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0  

Frame :  106
Size :  640  X  480
[[   0    0    0 ... 1866 1866 1866]
 [   0    0    0 ... 1861 1861 1861]
 [   0    0    0 ... 1850 1850 1850]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  107
Size :  640  X  480
[[   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0 1960]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  108
Size :  640  X  480
[[   0    0    0 ... 2037 2037 2037]
 [   0    0    0 ... 2017 2017 2017]
 [   0    0    0 ... 1991 1991 1991]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  109
Size :  640  X  480
[[   0    0    0 ... 1796 1796 1823]
 [   0    0    0 ... 1796 1796 1823]
 [   0    0    0 ... 1786 1786 1807]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0   

Size :  640  X  480
[[   0    0    0 ... 1906 1906 1906]
 [   0    0    0 ... 1906 1906 1906]
 [   0    0    0 ... 1900 1900 1900]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  141
Size :  640  X  480
[[   0    0    0 ... 1906 1906 1906]
 [   0    0    0 ... 1906 1906 1906]
 [   0    0    0 ... 1900 1900 1900]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  142
Size :  640  X  480
[[   0    0    0 ... 1906 1906 1906]
 [   0    0    0 ... 1883 1883 1883]
 [   0    0    0 ... 1861 1861 1861]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  143
Size :  640  X  480
[[   0    0    0 ... 1807 1801 1801]
 [   0    0    0 ... 1801 1796 1796]
 [   0    0    0 ... 1801 1796 1796]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [  

...

Frame :  1024
Size :  640  X  480
[[   0    0    0 ... 1833 1844 1844]
 [   0    0    0 ... 1833 1844 1844]
 [   0    0    0 ... 1844 1850 1850]
 ...
 [   0    0    0 ...    0    0    0]
 [ 554  554  554 ...    0    0    0]
 [ 553  553  553 ...    0    0    0]]

Frame :  1025
Size :  640  X  480
[[   0    0    0 ... 2057    0    0]
 [   0    0    0 ... 2044 2044 2044]
 [   0    0    0 ... 2037 2037 2030]
 ...
 [   0    0    0 ...    0    0    0]
 [ 556  556    0 ...    0    0    0]
 [ 556  556  556 ...    0    0    0]]

Frame :  1026
Size :  640  X  480
[[   0    0    0 ... 1761 1761 1766]
 [   0    0    0 ... 1761 1761 1766]
 [   0    0    0 ... 1776 1776 1786]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [ 556  556    0 ...    0    0    0]]

Frame :  1027
Size :  640  X  480
[[   0    0    0 ... 1817 1817 1823]
 [   0    0    0 ... 1817 1817 1823]
 [   0    0    0 ... 1833 1844 1844]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    

Frame :  1056
Size :  640  X  480
[[65535 65535     0 ...  1895  1895  1895]
 [65535 65535 65535 ...  1895  1895  1895]
 [65535 65535 65535 ...  1906  1906  1906]
 ...
 [    0   567   567 ...     0     0     0]
 [  568   567   567 ...     0     0     0]
 [  568   568   569 ...     0     0     0]]

Frame :  1057
Size :  640  X  480
[[   0    0    0 ...    0    0 1948]
 [   0    0    0 ...    0    0 1948]
 [   0    0    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [ 565    0    0 ...    0    0    0]
 [ 564  564  564 ...    0    0    0]]

Frame :  1058
Size :  640  X  480
[[   0    0    0 ... 1966 1966 1960]
 [   0    0    0 ... 1966 1966 1960]
 [   0    0    0 ... 1966 1966 1966]
 ...
 [ 572  571  571 ...    0    0    0]
 [ 571  570  570 ...    0    0    0]
 [ 570  570  570 ...    0    0    0]]

Frame :  1059
Size :  640  X  480
[[   0    0    0 ... 1828 1833 1833]
 [   0    0    0 ... 1828 1833 1833]
 [   0    0    0 ... 1828 1839 1839]
 ...
 [   0    0    0 ...    0

Size :  640  X  480
[[    0     0 65535 ...  1924  1930  1930]
 [    0 65535 65535 ...  1918  1924  1924]
 [    0 65535 65535 ...  1912  1912  1912]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]

Frame :  1088
Size :  640  X  480
[[    0     0 65535 ...  1900  1900  1900]
 [    0     0 65535 ...  1900  1900  1900]
 [    0     0 65535 ...  2030  2030  1966]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]

Frame :  1089
Size :  640  X  480
[[   0    0    0 ... 1991 1991 1991]
 [   0    0    0 ... 1966 1979 1979]
 [   0    0    0 ... 1948 1960 1960]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  1090
Size :  640  X  480
[[   0    0    0 ... 1985 1985 1985]
 [   0    0    0 ... 1966 1972 1972]
 [   0    0    0 ... 1954 1960 1960]
 ...
 [ 

Frame :  1119
Size :  640  X  480
[[   0    0    0 ... 1930 1930 1930]
 [   0    0    0 ... 1912 1912 1912]
 [   0    0    0 ... 1895 1900 1900]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  1120
Size :  640  X  480
[[   0    0    0 ... 2050    0    0]
 [   0    0    0 ... 2050 2071 2071]
 [   0    0    0 ... 2064 2071 2071]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]

Frame :  1121
Size :  640  X  480
[[   0    0    0 ... 1877 1877 1877]
 [   0    0    0 ... 1872 1872 1872]
 [   0    0    0 ... 1872 1872 1872]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [ 536    0    0 ...    0    0    0]]

Frame :  1122
Size :  640  X  480
[[   0    0    0 ... 1979 1979 1979]
 [   0    0    0 ... 1979 1979 1979]
 [   0    0    0 ... 1998 1998 1998]
 ...
 [ 535  535  535 ...    0    0    0]
 [ 535  535  535 ...    

Frame :  1154
Size :  640  X  480
[[ 721  721  721 ... 1889 1889 1889]
 [ 721  721  721 ... 1883 1883 1883]
 [ 721  721  721 ... 1883 1883 1883]
 ...
 [   0    0    0 ...    0    0    0]
 [ 687  688  689 ...    0    0    0]
 [ 687  688  689 ...    0    0    0]]

Frame :  1155
Size :  640  X  480
[[ 720  720  720 ... 1872 1877 1877]
 [ 720  720  720 ... 1872 1877 1877]
 [ 720  720  721 ... 1872 1877 1877]
 ...
 [ 684    0    0 ...    0    0    0]
 [ 684  684    0 ...    0    0    0]
 [ 684  684  685 ...    0    0    0]]

Frame :  1156
Size :  640  X  480
[[ 718  718  718 ...    0    0    0]
 [ 718  718  718 ...    0    0 1966]
 [ 718  718  718 ... 1942 1942 1942]
 ...
 [   0    0    0 ...    0    0    0]
 [ 680  680  680 ...    0    0    0]
 [ 680  680  680 ...    0    0    0]]

Frame :  1157
Size :  640  X  480
[[ 717  717  717 ...    0    0    0]
 [ 717  717  717 ...    0    0    0]
 [ 717  717  718 ... 2085 2064 2064]
 ...
 [ 686    0    0 ...    0    0    0]
 [ 686  686  686 ...    